In [1]:
import pandas as pd

**Loading and Exploring the dataset**

In [5]:
file_path = 'data/raw/agri_app_base_dataset.xls'
data = pd.read_excel(file_path)

# Display the first few rows
print(data.head(10))
print(data.shape)


   Dist Code  Year  State Code    State Name Dist Name  RICE AREA (1000 ha)  \
0          1  1990          14  Chhattisgarh      Durg           397.899994   
1          1  1991          14  Chhattisgarh      Durg           393.200012   
2          1  1992          14  Chhattisgarh      Durg           398.399994   
3          1  1993          14  Chhattisgarh      Durg           410.200012   
4          1  1994          14  Chhattisgarh      Durg           430.100006   
5          1  1995          14  Chhattisgarh      Durg           424.000000   
6          1  1996          14  Chhattisgarh      Durg           407.100006   
7          1  1997          14  Chhattisgarh      Durg           432.799988   
8          1  1998          14  Chhattisgarh      Durg           411.899994   
9          1  1999          14  Chhattisgarh      Durg           457.940002   

   RICE PRODUCTION (1000 tons)  RICE YIELD (Kg per ha)  \
0                   481.399994                  1210.0   
1             